# SQL Country Club Study 
## Part 2 - Using SQLite

In [1]:
# Import libraries
import sqlite3
import pandas as pd

In [2]:
# Connect to the database
dbfile = 'sqlite_db_pythonsqlite.db'
con = sqlite3.connect(dbfile, uri = True) #opens connection in read only mode

In [3]:
# Check that the connection worked by checking on the database
cur = con.cursor()

print('Database contains tables: ', cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()) # Tables contained in the database
print('Bookings table columns: ', cur.execute('SELECT * FROM Bookings').description)
print('Facilities table columns: ', cur.execute('SELECT * FROM Facilities').description)
print('Members table columns: ', cur.execute('SELECT * FROM Members').description)

Database contains tables:  [('Bookings',), ('Facilities',), ('Members',)]
Bookings table columns:  (('bookid', None, None, None, None, None, None), ('facid', None, None, None, None, None, None), ('memid', None, None, None, None, None, None), ('starttime', None, None, None, None, None, None), ('slots', None, None, None, None, None, None))
Facilities table columns:  (('facid', None, None, None, None, None, None), ('name', None, None, None, None, None, None), ('membercost', None, None, None, None, None, None), ('guestcost', None, None, None, None, None, None), ('initialoutlay', None, None, None, None, None, None), ('monthlymaintenance', None, None, None, None, None, None))
Members table columns:  (('memid', None, None, None, None, None, None), ('surname', None, None, None, None, None, None), ('firstname', None, None, None, None, None, None), ('address', None, None, None, None, None, None), ('zipcode', None, None, None, None, None, None), ('telephone', None, None, None, None, None, None), 

Seems to be working. Now on to solve the questions! 

### Q10:
Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [5]:
sql_q10 = """
WITH totalrevenue AS (SELECT Facilities.name AS facility,
	SUM(CASE WHEN memid = 0 THEN slots * guestcost ELSE slots * membercost END) AS revenue
FROM Bookings
LEFT JOIN Facilities USING (facid)
GROUP BY facility)

SELECT facility, revenue
FROM totalrevenue
WHERE revenue < 1000
ORDER BY revenue
;
"""

pd.read_sql_query(sql_q10, con)

,facility,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


### Q11:
Produce a report of members and who recommended them in alphabetic surname, firstname order

In [7]:
sql_q11 = """
SELECT m1.firstname, m1.surname, (m2.firstname || ' ' || m2.surname) AS recommended_by
FROM Members AS m1
LEFT JOIN Members AS m2
ON m1.recommendedby = m2.memid
ORDER BY m1.surname, m1.firstname
;
"""
pd.read_sql_query(sql_q11, con)

,firstname,surname,recommended_by
0,Florence,Bader,Ponder Stibbons
1,Anne,Baker,Ponder Stibbons
2,Timothy,Baker,Jemima Farrell
3,Tim,Boothe,Tim Rownam
4,Gerald,Butters,Darren Smith
5,Joan,Coplin,Timothy Baker
6,Erica,Crumpet,Tracy Smith
7,Nancy,Dare,Janice Joplette
8,David,Farrell,None
9,Jemima,Farrell,None


### Q12:
Find the facilities with their usage by member, but not guests

In [12]:
sql_q12 = """
SELECT name, SUM(slots) AS slots_used
FROM Bookings INNER JOIN Facilities USING (facid)
WHERE memid <> 0
GROUP BY name
ORDER BY slots_used DESC
;
"""

pd.read_sql_query(sql_q12, con)

,name,slots_used
0,Badminton Court,1086
1,Tennis Court 1,957
2,Massage Room 1,884
3,Tennis Court 2,882
4,Snooker Table,860
5,Pool Table,856
6,Table Tennis,794
7,Squash Court,418
8,Massage Room 2,54


### Q13: 
Find the facilities usage by month, but not guests

In [21]:
sql_q13 = """
SELECT name, 
	strftime('%m', starttime) AS month,
    SUM(slots) AS slots_used
FROM Bookings INNER JOIN Facilities USING (facid)
WHERE memid <> 0
GROUP BY name, month
;
"""

print(pd.read_sql_query(sql_q13, con))

               name month  slots_used
0   Badminton Court    07         165
1   Badminton Court    08         414
2   Badminton Court    09         507
3    Massage Room 1    07         166
4    Massage Room 1    08         316
5    Massage Room 1    09         402
6    Massage Room 2    07           8
7    Massage Room 2    08          18
8    Massage Room 2    09          28
9        Pool Table    07         110
10       Pool Table    08         303
11       Pool Table    09         443
12    Snooker Table    07         140
13    Snooker Table    08         316
14    Snooker Table    09         404
15     Squash Court    07          50
16     Squash Court    08         184
17     Squash Court    09         184
18     Table Tennis    07          98
19     Table Tennis    08         296
20     Table Tennis    09         400
21   Tennis Court 1    07         201
22   Tennis Court 1    08         339
23   Tennis Court 1    09         417
24   Tennis Court 2    07         123
25   Tennis 

In [22]:
# Close connection
con.close()